# Alignment-based Measure Events

In this notebook, we adapt the alignment-based measure with focus on events that was introduced by van der Aalst et al. in 2012  in the paper: "Replaying history on process models for conformance checking and performance analysis" (doi: https://doi.org/10.1002/widm.1045). <br>
The measure is defined as follows:
<br>
For event log $E$, set of unique activities $E'$, process model $M$, $|\text{sim}(e)|$ as the number of times a state $s$ was visited by the log and $|\text{diff}(e)|$ as the number of unique activities when leaving state $s$, generalization is defined as:

$$Generalization(E,M) = 1-\frac{1}{|E|}\sum_{e\,\text{in}\,E'}|e\,\text{in}\,E|\,*\,{pnew}(|\text{diff}(e)|,|\text{sim}(e)|),\; \text{where} $$
$$pnew(w,n)\begin{cases}
    \frac{w(w+1)}{n(n-1)} & \text{if } n\geq w+2, \\
    1 & \text{otherwise}.
\end{cases}$$ 

If generalization converges towards $0$, it is highly likely that a new event occurs in state $s$, and if generalization converges towards $1$, it is highly unlikely.

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
from ocpa.objects.log.importer.ocel import factory as ocel_import_factory
from ocpa.algo.discovery.ocpn import algorithm as ocpn_discovery_factory
from src.models.alignment_measure import alignment_measure, filter_case_variants
from src.utils import get_happy_path_log, create_flower_model
from ocpa.visualization.log.variants import factory as variants_visualization_factory
from ocpa.objects.log.importer.csv import factory as ocel_import_factory_csv

# O2C Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [7]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [8]:
value = alignment_measure(ocel,ocpn)
value

1.0

### Happy Path Petri Net

In [9]:
happy_path__ocel = get_happy_path_log(filename)

In [10]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [11]:
value = alignment_measure(ocel,happy_path_ocpn)
value

1.0

### Flower Model Petri Net

In [12]:
ots = ["order","item","delivery"]

In [13]:
flower_ocpn = create_flower_model(filename,ots)

In [14]:
value = alignment_measure(ocel,flower_ocpn)
value

1.0

# P2P Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [15]:
filename = "../src/data/jsonocel/p2p-normal.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [16]:
value = alignment_measure(ocel,ocpn)
value

0.9997

### Happy Path Petri Net

In [17]:
happy_path__ocel = get_happy_path_log(filename)

In [18]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [19]:
value = alignment_measure(ocel,happy_path_ocpn)
value

0.9997

### Flower Model Petri Net

In [23]:
ots = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]

In [24]:
flower_ocpn = create_flower_model(filename,ots)

In [25]:
value = alignment_measure(ocel,flower_ocpn)
value

0.9998

# BPI-Challenge 2017 Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [26]:
filename = "../src/data/jsonocel/BPI2017-Final.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [27]:
value = alignment_measure(ocel,ocpn)
value

1.0

### Happy Path Petri Net

In [28]:
happy_path__ocel = get_happy_path_log(filename)

In [29]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [30]:
value = alignment_measure(ocel,happy_path_ocpn)
value

1.0

### Flower Model Petri Net

In [31]:
ots = ["application","offer"]

In [32]:
flower_ocpn = create_flower_model(filename,ots)

In [33]:
value = alignment_measure(ocel,flower_ocpn)
value

1.0

# Measure with each variant appearing only once in log

In a next step, we also want to see if our logs behave in a similar way, as soon as we use a log for the generated models that consists of each variant appearing only once. In a first step, we need to generate the logs for these filtered subsets and save them as csv (is saved as pandas df after filtering). With this csv, we are able to import the filtered log as ocel log again and calculate the results for our alignment measure. We need to be aware that we do not generate new ocpn, but keep the ones already generated with the full log.

# O2C Log

In [4]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [5]:
filter_case_variants (ocel, 'event_variant','event_id', "../src/data/filtered_traces/order_process_filtered.csv")

In [6]:
filename_filtered = "../src/data/filtered_traces/order_process_filtered.csv"
object_types = ["order","item","delivery"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_filtered = ocel_import_factory_csv.apply(file_path=filename_filtered, parameters=parameters)

In [7]:
print("Number of process executions: "+str(len(ocel_filtered.process_executions)))
print("Number of variants: "+str(len(ocel_filtered.variants)))

Number of process executions: 12
Number of variants: 12


### Standard Petri Net

In [8]:
value = alignment_measure(ocel_filtered,ocpn)
value

0.9999

### Happy Path Petri Net

In [9]:
happy_path__ocel = get_happy_path_log(filename)

In [10]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [11]:
value = alignment_measure(ocel_filtered,happy_path_ocpn)
value

1.0

### Flower Model Petri Net

In [12]:
ots = ["order","item","delivery"]

In [13]:
flower_ocpn = create_flower_model(filename,ots)

In [14]:
value = alignment_measure(ocel_filtered,flower_ocpn)
value

1.0

# P2P Log

In [15]:
filename = "../src/data/jsonocel/p2p-normal.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [16]:
filter_case_variants (ocel, 'event_variant','event_id', "../src/data/filtered_traces/p2p-normal_filtered.csv")

In [17]:
filename_filtered = "../src/data/filtered_traces/p2p-normal_filtered.csv"
object_types = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_filtered = ocel_import_factory_csv.apply(file_path=filename_filtered, parameters=parameters)

In [18]:
print("Number of process executions: "+str(len(ocel_filtered.process_executions)))
print("Number of variants: "+str(len(ocel_filtered.variants)))

Number of process executions: 20
Number of variants: 20


### Standard Petri Net

In [19]:
value = alignment_measure(ocel_filtered,ocpn)
value

0.9947

### Happy Path Petri Net

In [20]:
happy_path__ocel = get_happy_path_log(filename)

In [21]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [22]:
value = alignment_measure(ocel_filtered,happy_path_ocpn)
value

0.9947

### Flower Model Petri Net

In [23]:
ots = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]

In [24]:
flower_ocpn = create_flower_model(filename,ots)

In [25]:
value = alignment_measure(ocel_filtered,flower_ocpn)
value

0.9965

# BPI-Challenge 2017 Log

In [26]:
filename = "../src/data/jsonocel/BPI2017-Final.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [27]:
filter_case_variants (ocel, 'event_variant','event_id', "../src/data/filtered_traces/BPI2017-Final_filtered.csv")

In [ ]:
filename_filtered = "../src/data/filtered_traces/BPI2017-Final_filtered.csv"
object_types = ["application","offer"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_filtered = ocel_import_factory_csv.apply(file_path=filename_filtered, parameters=parameters)

In [ ]:
print("Number of process executions: "+str(len(ocel_filtered.process_executions)))
print("Number of variants: "+str(len(ocel_filtered.variants)))

### Standard Petri Net

In [ ]:
value = alignment_measure(ocel_filtered,ocpn)
value

### Happy Path Petri Net

In [ ]:
happy_path__ocel = get_happy_path_log(filename)

In [ ]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [ ]:
value = alignment_measure(ocel_filtered,happy_path_ocpn)
value

### Flower Model Petri Net

In [ ]:
ots = ["application","offer"]

In [ ]:
flower_ocpn = create_flower_model(filename,ots)

In [ ]:
value = alignment_measure(ocel_filtered,flower_ocpn)
value